In [1]:
import os
import shutil
from base64 import b64encode
from http.client import HTTPSConnection
import yaml
import requests
import pandas as pd
import os
import re
import gensim.models
import copy
import numpy as np
import operator
import json
import math

In [2]:
'''
Orginally I wanted to save the user/pass as an enviromental variable. However, since
jupyterlab is called as a service, it doesn't load .bashrc therefore, it's not
possible to call enviro values while running in juypterlab. However, the end goal
is to run this script outside of jupyter to gather recipes and push to github. 
So getting vars from .bashrc should work. 
'''

#Try to get from enviro var. Need to check if this works when ran outside jupyter
email = os.environ.get('PAPRIKAAPPEMAIL')
password = os.environ.get('PAPRIKAAPPPASSWORD')

#If email or password wasn't found, read bashrc directly and find the vars
if email == None or password == None:
    print('Brute force - read .bashrc and get user/pass')
    f = open("/home/pi/.bashrc", "r")
    lines = f.readlines()

    for line in lines:
        matchEmail = re.match( r'(export PAPRIKAAPPEMAIL=)(.*)', line, re.MULTILINE)
        if matchEmail:
            email = matchEmail.group(2)

        matchPassword = re.match( r'(export PAPRIKAAPPPASSWORD=)(.*)', line, re.MULTILINE)
        if matchPassword:
            password = matchPassword.group(2)


c = HTTPSConnection("www.paprikaapp.com")
userAndPass = b64encode(bytes(email+":"+password, 'utf-8')).decode("ascii")
headers = { 'Authorization' : 'Basic %s' %  userAndPass }

c.request('GET', '/api/v1/sync/categories/', headers=headers)
res = c.getresponse()
data = res.read()
categories = {}
for item in json.loads(data)['result']:
    categories[item['uid']] = item['name']

c.request('GET', '/api/v1/sync/recipes/', headers=headers)
res = c.getresponse()
data = res.read()
#print(data)

df = pd.DataFrame()

recipes = []
for item in json.loads(data)['result']:
    c.request('GET', '/api/v1/sync/recipe/'+item['uid']+'/', headers=headers)
    res = c.getresponse()
    data = res.read()
    recipe = json.loads(data)['result']
    
    #Check to see if it's in the Drinks catergory
    if 'F59B5D0D-9C8A-41FD-8134-F26921D1B19A-18063-0000098A8D94DC39' in recipe['categories']:
        #print(recipe)
        
        df = df.append(recipe, ignore_index=True)


Brute force - read .bashrc and get user/pass


In [3]:
df.dtypes

categories           object
cook_time            object
created              object
description          object
difficulty           object
directions           object
hash                 object
image_url            object
in_trash            float64
ingredients          object
is_pinned           float64
name                 object
notes                object
nutritional_info     object
on_favorites        float64
on_grocery_list      object
photo                object
photo_hash           object
photo_large          object
photo_url            object
prep_time            object
rating              float64
scale                object
servings             object
source               object
source_url           object
total_time           object
uid                  object
dtype: object

In [4]:
import re
df[ (df['ingredients'].str.contains(r'(gin)', flags=re.IGNORECASE)) & (df['ingredients'].str.contains(r'(lemon|lime)', flags=re.IGNORECASE))  ]['ingredients']

/home/pi/.local/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


3     2 ounces gin\n3/4 ounce sweet vermouth\n3/4 ou...
4     1 1/2 ounces (45ml) bourbon\n2 ounces (60ml) d...
6       2 oz gin\n1 Oz orgeat\n.75 lemon juice\nBitters
13    4 oz. Ginger Beer\n1½ oz. Bourbon\n½ oz. Black...
23    .25 oz freshly squeezed lemon juice\n3-4 mint ...
28    1 1/2 ounces (45ml) Cynar\n1 1/2 ounces (45ml)...
31    2 oz gin\n1 tablespoons honey\n2 tablespoons f...
32    ice\n1 ounce gin (Tanqueray or Beefeater recom...
33    Small handful of blackberries\nMint\n1-1.5 oz ...
34    12 blackberries\n20 fresh mint leaves\n2 Meyer...
36    2 oz pear infused vodka, recipe below\n1/2 oz ...
38    1.5 oz. NOLET’S Silver Gin\n1 oz. Fresh Lime J...
41    1 lemon\n3 tablespoons (1 1/2 ounces) gin\n1 1...
42    1.5 oz. NOLET’S Silver Gin\n1 oz. Lemon Juice\...
44    2 ounces gin\n1/2 ounce lemon juice\n1/2 ounce...
46    2 ounces London dry gin\n3/4 ounces triple sec...
47    Adapted from Bon Appetit\nFor 2 tall drinks\n1...
51    3 mint springs\n1 and ½ ounce gin\n1/2 oun

In [5]:
#Export all recipes
jsonRecipeExport = df.to_json(orient="index")

with open('recipes.json', 'w') as outfile:
    outfile.write(jsonRecipeExport)

In [6]:
#By default, ingredients are seperated by \n. Replace with space
#df['ingredients'] = df['ingredients'].str.replace('\n', '-')

#Need to clean up ingredients
listOfIngredients = df['ingredients'].tolist()

In [7]:
#Returns regex line \b<term1>\b|\b<terms2>...
def regexOrList(listOfTerms):
    returnStr = ''
    for term in listOfTerms:
        tempTerm = '\\b' + term + '\\b' + '|'
        returnStr = returnStr + tempTerm
        
    #Remove last |
    returnStr = returnStr[:-1]
    
    return returnStr

In [8]:
#'Flatten list to string'
ingredientList = ' '
ingredientList = ingredientList.join(listOfIngredients)

#I believe the last item per recipe doesn't have a \n, so 
# it ends up with WordAWordB. Need to fix
ingredientList = ingredientList.split('\n')

cleanedIngredientsList = []

for _line in ingredientList:
    #print(_line)
    
    #Clean out numbers
    regexNumbers = re.compile(r'(\d/\d)|(\d)|(\.\d*)|(¾)|(½)|(¼)', re.IGNORECASE)
    _line = re.sub(regexNumbers, '', _line)
    
    #Clean out measurements
    measurementsList = ['ounce', 'ounces', 'oz', 'oz.', 'ml', 'mL', 'g', 'gram', 'grams', 'tablespoon', 'tablespoons',
                       'tsp', 'tbsp', 'teaspoon', 'teaspoons', 'part', 'parts', 'cup', 'cups', 'barspoons']
    regexMeasurements = re.compile(regexOrList(measurementsList), re.IGNORECASE)
    _line = re.sub(regexMeasurements, '', _line)
    
    
    #Single words or characters - Ignore case
    singleWordList = ['freshly', 'garnish', 'glass', 'glasses', 'and', 'few', 'about', 'dash', 'dashes',
                      'or', 'ice', 'cube', 'your', 'favorite', 'other', 'good-quality', 'optional', 'to', 'from',
                      'the', 'for', 'with', 'a', 'coup', 'Coupe', 'cubes', 'tall', 'drinks']
    regexSingleWords = re.compile(regexOrList(singleWordList), re.IGNORECASE)
    _line = re.sub(regexSingleWords, '', _line)

    #Anything in quotes
    regexQuotes = re.compile(r'\(.*\)', re.IGNORECASE)
    _line = re.sub(regexQuotes, '', _line)

    #Remove special characters
    regexSpecialChars = re.compile(r'\:|,|\*|-|\(|\)|&|\*|\%', re.IGNORECASE)
    _line = re.sub(regexSpecialChars, '', _line)
    
    #Remove \r \n
    _line = _line.strip()
    
    #Remove any extra spaces
    _line = re.sub("\s\s+", " ", _line)
    
    #Remove lines that are too short or empty
    if len(_line) < 2:
        pass
    elif _line.count(' ') > 3:
        pass
    else:
        #print(_line)
        cleanedIngredientsList.append(_line)
        pass

        
print(cleanedIngredientsList)

['bourbon rye whiskey', 'Cointreau orange liqueur', 'sweet vermouth', 'squeezed lemon juice', 'maraschino liqueur', 'grapefruit juice', 'lime juice', 'simple syrup', 'grapefruit juice', 'lemon juice', 'Campari', 'simple syrup gin', 'sweet vermouth', 'dry vermouth', 'orange bitters bourbon', 'dry hard cider', 'ginger beer', 'Angostura bitters', 'Fresh lemon juice', 'Simple syrup', 'Huckleberry Shrub', 'Egg white', 'Huckleberries', 'Cider vinegar', 'Sugar gin', 'orgeat', 'lemon juice', 'Bitters bourbon', 'Averna amaro', 'Punt e Mes vermouth', 'Kahlua Coffee Liqueur', 'fresh grapefruit juice', 'fresh lime juice', 'Cholula hot sauce', 'Flaky salt white rum', 'orange curaçao', 'lime juice squeezed', 'orgeat', 'dark rum', 'lime wheel', 'Campari', 'Lemon Juice', 'Orange Juice', 'Espresso', 'Dulce de Leche', 'pineapple juice', 'vodka', 'grenadine Ginger Beer', 'Bourbon', 'Blackberry Cordial', 'Lime Juice', 'cherry liqueur', 'honey syrup CARAMELIZED HONEY', 'handful fresh thyme', 'water', 'hone

In [9]:
#Need to check for duplicates
duplicates = set([x for x in cleanedIngredientsList if cleanedIngredientsList.count(cleanedIngredientsList) > 1])
print(duplicates)

set()


In [10]:
class categorizeObj():
    def __init__(self, model):
        #List of totals. Always append to list
        self.categoriesList = { 
            'alcohol' : [],
            'liqueur' : [],
            'fruit' : [],
            'savory' : [],
            'herb' : [],
            'sweeteners' : []
        }
        
        #Duplicate dict to store totals - Need to set default to 'unknown' category
        self.categoriesTotals = copy.deepcopy(self.categoriesList)
        for k, v in self.categoriesTotals.items():
            self.categoriesTotals[k] = 'unknown'
        
        self.model = model
        
        
    #Find similarity and add to total as list!
    def processWord(self, word):
        
        #Verify Word is in model
        word_vectors = model.wv
        if word in word_vectors.vocab:
            #print('Word Found: ', word)
            #Loop through dict and add to dictionary
            for key, value in self.categoriesList.items():
                self.categoriesList[key].append( model.wv.similarity(key, word) )
        else:
            #print(word, 'not found!')
            pass
    
    
    def processList(self, listOfWords):
        for word in listOfWords:
            self.processWord(word)
    
    
    #Chrunch the numbers
    def categorizeSentence(self, mathType='average'):
        if mathType == 'average':
            for key, values in self.categoriesList.items():
                #print(key, values)
                self.categoriesTotals[key] = np.average( values )
                
        elif mathType == 'median':
            for key, values in self.categoriesList.items():
                self.categoriesTotals[key] = np.median( values )
                
        elif mathType == 'max':
            for key, values in self.categoriesList.items():
                self.categoriesTotals[key] = np.max( values )
        else:
            print('Selection not valid')
    
    
    #This function really shows how bad the NN is. Unsure of the reason.
    def categorize(self):
        #Duplicate to make manual modification
        categoriesTotalsModified = copy.deepcopy(self.categoriesTotals)
        
        #Combine Spices & Herbs into single category and compare against others
        categoriesTotalsModified['spices_and_herbs'] = (categoriesTotalsModified['savory'] + categoriesTotalsModified['herb'])/2.0
        del categoriesTotalsModified['savory']
        del categoriesTotalsModified['herb']
           
        #Spices and Herbs still throws off everything. Artifically adjust by lowering the score
        categoriesTotalsModified['alcohol'] = categoriesTotalsModified['alcohol'] * 0.85
        categoriesTotalsModified['liqueur'] = categoriesTotalsModified['liqueur'] * 0.92
        categoriesTotalsModified['fruit'] = categoriesTotalsModified['fruit'] * 0.85
        categoriesTotalsModified['spices_and_herbs'] = categoriesTotalsModified['spices_and_herbs'] * 0.6
        categoriesTotalsModified['sweeteners'] = categoriesTotalsModified['sweeteners'] * 0.8

        #print(categoriesTotalsModified)
        
        #Also return a value. Alcohol or Liquor: 1, fruit and spice_herb:2, sweeteners: 3, unknown: 4

        #Return format will be (category, value)
        try:
            if math.isnan(categoriesTotalsModified['alcohol']):
                return ('unknown', 40)
            
            #Get the max of the modified scores and return
            maxValue = max(categoriesTotalsModified, key=lambda key: categoriesTotalsModified[key])
            
            if maxValue == 'alcohol' or maxValue == 'liqueur':
                value = 10
            elif maxValue == 'fruit' or maxValue == 'spices_and_herbs':
                value = 20
            elif maxValue == 'sweeteners':
                value = 30
            else:
                maxValue = 'unknown'
                value = 50
                
            return (maxValue, value)
        except:
            return ('unknown', 40)
    

In [11]:
'''End goal is to put each ingredient into a bucket: base-spirit:1, liqueur:2, citrus:3,
   fruit:4, spices:6, sweet:5, or dairy:7 

The final format: {"name": "Gin", "value": 1, category: "Base-spirit"}

Each element can be multiple words (a sentence). Each word can be an alcohol, liqueur, citrus, etc.
 The idea is to average(?) the values from the models and decides on a single catergory
'''

#OutputFile
ingredientsKeyWords = []

#Load model
model = gensim.models.Word2Vec.load('alcoholAI.model')

#Loop through cleanedIngredientsList and catergorize
for ingredients in cleanedIngredientsList:

    #Init the model
    category = categorizeObj(model)
    
    #Split into list and make lowercase
    ingredientsModified = ingredients.split(' ')
    ingredientsModified = [x.lower() for x in ingredientsModified]
    
    #Split by space and get list
    category.processList( ingredientsModified )
    
    #Crunch the numbers
    category.categorizeSentence(mathType='average')
    
    returnVals = category.categorize()
    
    #print(ingredientsModified, returnVals, '\n')
    
    toAdd = {}
    toAdd['x'] = str( ingredients )
    toAdd['value'] = str( 1 )
    toAdd['category'] = str( returnVals[0] )
    
    ingredientsKeyWords.append(toAdd)

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
#Need to clean out the recipies - Garbage in == Garbage out

In [13]:
#Export keywords - Need to remove [] from file
with open('keywords.json', 'w') as outfile:
    json.dump(ingredientsKeyWords, outfile)

In [14]:
'''
End goal is to make another netilfy page. No flask

Need to load the username and password from enviromental variables
https://www.nylas.com/blog/making-use-of-environment-variables-in-python/

Create a python script to export a json object with all recipes.
Use code above - export json to file

The rest is all html/javascript/bootstrap
 - Word cloud where clicking on word will add AND term to search
 - All words are from the ingredients
 


'''

'\nEnd goal is to make another netilfy page. No flask\n\nNeed to load the username and password from enviromental variables\nhttps://www.nylas.com/blog/making-use-of-environment-variables-in-python/\n\nCreate a python script to export a json object with all recipes.\nUse code above - export json to file\n\nThe rest is all html/javascript/bootstrap\n - Word cloud where clicking on word will add AND term to search\n - All words are from the ingredients\n \n\n\n'